In [2]:
import pandas as pd
import json
import re

class Formatter:
    def __init__(self, input_file, output_file, clean_text=True):
        self.input_file = input_file
        self.output_file = output_file
        self.clean_text = clean_text

    def clean(self, text):
        if not isinstance(text, str):
            return text
        # Loại bỏ xuống dòng, HTML tags nếu có, và chuẩn hoá khoảng trắng
        text = re.sub(r'\s+', ' ', text)                    # chuẩn hoá khoảng trắng
        text = re.sub(r'<.*?>', '', text)                   # loại bỏ thẻ HTML
        text = re.sub(r'[^\x00-\x7F\u00C0-\u1EF9]+', ' ', text)  # loại emoji và ký tự đặc biệt ngoài Unicode cơ bản
        return text.strip()

    def format_data(self):
        # Load data từ file JSON
        with open(self.input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        df = pd.DataFrame(data)

        # Làm sạch văn bản nếu bật clean_text
        if self.clean_text:
            for col in ['title', 'content', 'name']:
                if col in df.columns:
                    df[col] = df[col].apply(self.clean)

        # Chuẩn hoá datetime
        df['date'] = pd.to_datetime(df['date'], utc=True)

        # Ghi ra file CSV
        df.to_csv(self.output_file, index=False, encoding='utf-8')

# formatter = Formatter("../data-raw/data-post-json.json", "../data/dum-data.csv")
# formatter.format_data()

In [7]:
df = pd.read_csv("../data/dum-data.csv")
print(df.head())

       id       name                       date  \
0  130300        PNJ  2025-03-27 18:42:09+00:00   
1    1727   giá vàng  2025-02-05 05:04:56+00:00   
2   42862  coca cola  2014-06-13 04:29:00+00:00   
3   15537     BeatVN  2021-10-12 00:50:49+00:00   
4    1583   giá vàng  2025-02-05 01:56:47+00:00   

                                               title  \
0  Mặt dây kim cương, ai tên H chốt liền dùm em  ...   
1  GIẬT MÌNH: Giá vàng hôm nay 5/2/2025: Bỏ râ...   
2  Coca-Cola in tên người tiêu dùng lên nhãn sản ...   
3  Cô Hằng cứ mơ nhiều thế này thì ... #beatvn #s...   
4    Giá vàng tiếp tục  leo thang  lên 91 triệu đồng   

                                             content  sentiment_score  
0  Mặt dây kim cương, ai tên H chốt liền dùm em  ...                1  
1  GIẬT MÌNH: Giá vàng hôm nay 5/2/2025: Bỏ râ...                0  
2  Hoạt động trên nằm trong chương trình  Trao Co...                1  
3  #BEATVN #BEATNEWS #Trendy #Shorts Các bạn đăng...                

In [12]:
df.describe(include='all')

,id,name,date,title,content,sentiment_score
count,231327.000000,231327,231312,231051,186415,231327.000000
unique,NaN,340,211403,221583,171759,NaN
top,NaN,hà,2025-03-07 08:00:00+00:00,Cổ Nhân Nói | Kts Phong Thủy Lưu Hà #ktsluuha ...,"Trong video này, chúng ta sẽ khám phá những câ...",NaN
freq,NaN,15997,108,492,295,NaN
mean,122964.132643,NaN,NaN,NaN,NaN,1.206258
std,71234.918770,NaN,NaN,NaN,NaN,0.802867
min,41.000000,NaN,NaN,NaN,NaN,0.000000
25%,61495.500000,NaN,NaN,NaN,NaN,1.000000
50%,123274.000000,NaN,NaN,NaN,NaN,1.000000
75%,181698.500000,NaN,NaN,NaN,NaN,2.000000


In [9]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231327 entries, 0 to 231326
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               231327 non-null  int64 
 1   name             231327 non-null  object
 2   date             231312 non-null  object
 3   title            231051 non-null  object
 4   content          186415 non-null  object
 5   sentiment_score  231327 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 10.6+ MB


Index(['id', 'name', 'date', 'title', 'content', 'sentiment_score'], dtype='object')

In [24]:
df.isna().sum()

id                     0
name                   0
date                  15
title                276
content            44912
sentiment_score        0
date_only             15
month                 15
dtype: int64

In [25]:
df[df.isna().any(axis=1)]

,id,name,date,title,content,sentiment_score,date_only,month
10,11776,PNJ,2025-02-18 08:24:31+00:00,Đi giao hàng cho khách nhưng ra dẻ suy #diam...,NaN,1,2025-02-18,2025-02
11,47762,tang diên,2025-03-08 09:20:05+00:00,Tang Diên,NaN,1,2025-03-08,2025-03
13,17998,Hải sản,2025-02-22 21:19:43+00:00,Chủ nhật 23/2/2025 cập nhật giá bán lẻ hải sản...,NaN,1,2025-02-22,2025-02
15,23741,kem đánh răng,2025-02-25 05:57:54+00:00,Kem đánh răng cho bé hữu cơ nuốt được #hangmom...,NaN,2,2025-02-25,2025-02
18,49050,kim sae ron,2025-02-17 17:41:27+00:00,Kim Sae Ron found dead at her home #speedson...,NaN,1,2025-02-17,2025-02
...,...,...,...,...,...,...,...,...
231093,244547,sự nghiệp chướng,2025-04-21 14:54:59+00:00,Sự Nghiệp Chướng / Pháo - cover Huấn Hoa Hồng ...,NaN,2,2025-04-21,2025-04
231141,245570,Mỹ Tâm,2025-04-21 10:13:33+00:00,Sao Việt 21/4: Hồng Nhung rạng rỡ dù đang trị ...,NaN,1,2025-04-21,2025-04
231203,241126,vinfast,2025-04-21 09:00:00+00:00,VinFast tung ưu đãi 6-70 triệu đồng cho khách ...,NaN,1,2025-04-21,2025-04
231298,245657,Thùy Tiên,2025-04-21 23:37:31+00:00,Sau cơn mưa Quang Linh vs hoa hậu Thùy Tiên ...,NaN,2,2025-04-21,2025-04


In [26]:
df.dropna(inplace=True)

In [27]:
df.isna().sum()

id                 0
name               0
date               0
title              0
content            0
sentiment_score    0
date_only          0
month              0
dtype: int64

In [28]:
df.shape

(186415, 8)

In [29]:
df['date'] = pd.to_datetime(df['date'], format='ISO8601', errors='coerce')

In [30]:
df[df['date'].isna()]

,id,name,date,title,content,sentiment_score,date_only,month


In [31]:
df['date_only'] = df['date'].dt.date
df['month'] = df['date'].dt.to_period('M')

C:\Users\thong\AppData\Local\Temp\ipykernel_10052\1601823848.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = df['date'].dt.to_period('M')


In [32]:
# Đảm bảo cột date là datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Lấy 3 ngày gần nhất (duy nhất, không lặp)
latest_3_days = df['date'].dropna().dt.date.sort_values(ascending=False).unique()[:3]

# Lọc các dòng thuộc 3 ngày đó
df_latest_3_days = df[df['date'].dt.date.isin(latest_3_days)]

df_latest_3_days.head(10)

,id,name,date,title,content,sentiment_score,date_only,month
206,243309,KFC,2025-04-21 09:00:18+00:00,KFC kiểu người lười #mukbang #kfc,#scary tiktok #kpoprandomdancemirroredtiktok #...,1,2025-04-21,2025-04
3094,244761,Phương Mỹ Chi,2025-04-21 17:17:19+00:00,GƯƠNG MẶT TRẺ VIỆT NAM TRIỂN VỌNG của tuii đó ...,GƯƠNG MẶT TRẺ VIỆT NAM TRIỂN VỌNG của tuii đó ...,1,2025-04-21,2025-04
3254,243550,Phương Mỹ Chi,2025-04-20 06:44:15+00:00,Kiểu tóc bánh lái của phụ nữ Nam Kỳ Việt Nam #...,Kiểu tóc bánh lái của phụ nữ Nam Kỳ Việt Nam #...,2,2025-04-20,2025-04
4933,242956,giá vàng,2025-04-21 06:25:41+00:00,Giá vàng hôm nay ngày 21/4/2025 - giá vàng 999...,Cảm ơn các bạn đã xem clip. Hãy SUBSCRIBE và C...,1,2025-04-21,2025-04
5255,236616,nhã phương,2025-04-20 09:41:27+00:00,Nhã Phương và Cuộc Chiến Với Nhan Sắc,Nhã Phương và Cuộc Chiến Với Nhan Sắc #maimedi...,2,2025-04-20,2025-04
9120,240711,giá vàng,2025-04-21 08:31:29+00:00,Giá vàng thế giới lập kỷ lục mới khi mối lo ng...,"Giá vàng giao ngay tăng 1% lên 3.361,53 USD/ o...",2,2025-04-21,2025-04
9139,240715,giá vàng,2025-04-21 03:00:55+00:00,Giá vàng khởi động tuần mới bật tăng 3 triệu đ...,Giá vàng mở cửa phiên đầu tuần tăng mạnh 3 tri...,2,2025-04-21,2025-04
11319,244787,giá vàng,2025-04-21 09:29:21+00:00,Tin Thế giới 21/4: Giá vàng tiếp tục tăng mạnh...,"VTC News | Tuần mới, nhiều thị trường quốc tế ...",2,2025-04-21,2025-04
11336,244786,giá vàng,2025-04-21 22:15:03+00:00,Giá vàng hôm nay 9999 ngày 22 tháng 4 năm 2025...,Giá vàng hôm nay 9999 ngày 22/4/2025- GIÁ VÀNG...,2,2025-04-21,2025-04
11355,242966,giá vàng,2025-04-21 10:40:49+00:00,Giá vàng hôm nay 9999 tối ngày 21/4/2025- GIÁ ...,Giá vàng hôm nay 9999 tối ngày 21/4/2025- GIÁ ...,2,2025-04-21,2025-04
